# Learning the electron density of water

## Pre-requisites

### IMPORTANT!

This notebook isn't yet fully runnable as it is required to have a specific
binary for the (not open-source) Quantum Chemistry code FHI-aims.

### Install the required packages

1. `pip install metatensor`
1. `pip install chemiscope`
1. `pip install
   git+https://github.com/luthaf/rascaline.git@b2cedfe870541e6d037357db58de1901eb116c41`




**NOTE**: this notebook has only been tested on the HPC system "jed". Due to
specific compilation of the quantum chemistry package `FHI-aims`, the Python
interface to running QC calculations may not be generalized to other operating
systems or hardware.

In [1]:
%load_ext autoreload
%autoreload 2

# Useful standard and scientific ML libraries
import os
import time
import ase.io
import matplotlib.pyplot as plt
import numpy as np
import py3Dmol
import torch

# M-Stack packages
import metatensor   # storage format for atomistic ML
import chemiscope  # interactive molecular visualization
import rascaline   # generating structural representations
from metatensor import Labels, TensorBlock, TensorMap
from rascaline.utils import clebsch_gordan

# Interfacing with FHI-aims
from rhocalc.aims import aims_calc, aims_parser

# Torch-based density leaning
from rholearn import io, data, loss, models, predictor
import settings

In [17]:
from settings import data_dir

# The absolute paths to 3 directories need to be defined, all as callables that
# take as input the structure index. They are defined in a hierarchical manner.

# The directory where the SCF calculation outputs will be saved
def scf_dir(A):
    return os.path.join(data_dir, f"{A}")

# The directory where the SCF restart will be read and RI calculation run (and
# outputs saved)
def ri_dir(A, restart_idx):
    return os.path.join(scf_dir(A), f"{restart_idx}")

# The directory where the RI calculation outputs will be processed from text
# files to metatensor format and saved.
def processed_dir(A, restart_idx):
    return os.path.join(ri_dir(A, restart_idx), "processed")

## Visualize structures in dataset

* Use `chemiscope`

In [10]:
from settings import idxs, frames, all_frames

chemiscope.show(
    frames,
    properties={
        "Mean O-H bond length, Angstrom": [np.mean([f.get_distance(0, 1), f.get_distance(0, 2)]) for f in frames],
        "H-O-H angle, degrees": [f.get_angle(1, 0, 2) for f in frames],
    },
)

ChemiscopeWidget(value='{"meta": {"name": " "}, "structures": [{"size": 3, "names": ["O", "H", "H"], "x": [0.0…

## Generate learning targets

* These are the RI coefficients of the HOMO eigenstate
* We generate these in 2 steps: 1) SCF and 2) RI fitting

#### 1. Converge SCF for each structure

In [7]:
# Import the settings needed to run FHI-aims
from settings import aims_path, base_aims_kwargs, scf_kwargs, ri_kwargs, sbatch_kwargs

# Build a dict of settings for each calculation (i.e. structure)
# IMPORTANT: zip() is used to pair up the structure index and the structure
calcs = {
    A: {"atoms": frame, "run_dir": scf_dir(A)} for A, frame in zip(idxs, frames)
}

# And the general settings for all calcs
aims_kwargs = base_aims_kwargs.copy()
aims_kwargs.update(scf_kwargs)

# Run the SCF in AIMS
aims_calc.run_aims_array(
    calcs=calcs,
    aims_path=aims_path,
    aims_kwargs=aims_kwargs,
    sbatch_kwargs=sbatch_kwargs,
    run_dir=scf_dir,
)

Submitted batch job 11396302


sbatch: [ESTIMATION] The estimated cost of this job is CHF 11.20
sbatch: [CAPPING]    All users of the account cosmo-erc have consumed 117.06 CHF
sbatch: [CAPPING]    In addition, based on queued and running jobs all users of the account cosmo-erc will consume up to 0.00 CHF
sbatch: [CAPPING]    Your username abbott has consumed 4.99 CHF
sbatch: [CAPPING]    In addition, based on queued and running jobs your username abbott will consume up to 0.00 CHF
sbatch: ╭──────────────────────────────┬─────────────┬─────────────┬─────────────╮
sbatch: │ [in CHF]                     │ Capping     │ Consumed    │ Queued      │
sbatch: ├──────────────────────────────┼─────────────┼─────────────┼─────────────┤
sbatch: │ username : abbott            │ 0           │ 5.0         │ 0.0         │
sbatch: ├──────────────────────────────┼─────────────┼─────────────┼─────────────┤
sbatch: │ account : cosmo-erc          │ 1,610.653   │ 117.1       │ 0.0         │
sbatch: ╰──────────────────────────────┴──────

In [8]:
# Running this code will ensure that all calculations have finished.
# Should take < 10 seconds (1 node, 10 cores on 'jed' HPC)
all_finished = False
while not all_finished:
    calcs_finished = []
    for A in idxs:
        aims_out_path = os.path.join(scf_dir(A), "aims.out")
        if os.path.exists(aims_out_path):
            with open(aims_out_path, "r") as f:
                calcs_finished.append("Leaving FHI-aims." in f.read())  # AIMS finished
        else:
            calcs_finished.append(False)
    all_finished = np.all(calcs_finished)

# Check SCF converged
converged = []
for A in idxs:
    aims_out_path = os.path.join(scf_dir(A), "aims.out")
    if os.path.exists(aims_out_path):
        with open(aims_out_path, "r") as f:
            converged.append(
                "Self-consistency cycle converged." in f.read()
            )  # calculation converged
    else:
        converged.append(False)
if np.all(converged):
    print("All SCF calculations converged!")
else:
    print(
        "Some SCF calculations did not converge. structure idxs: ",
        [A for i, A in enumerate(idxs) if not converged[i]],
    )

All SCF calculations converged!


#### 2. Perform RI fitting on the scalar field of interest (HOMO)

* First we need to identify the HOMO
* This can be done by parsing the Kohn Sham orbital information from the
  converged SCF calculation. 
* This info got written to file "ks_orbital_info.out" above by using the
  `ri_fit_write_ks_orb_info: True` keyword set in `scf_kwargs`.

In [11]:
for restart_idx in [0, 1]:  # HOMO, LUMO

    # Write KSO weights for just the HOMO to file
    for A in idxs:
        # Parse the Kohn-Sham
        ks_info = aims_parser.get_ks_orbital_info(
            os.path.join(scf_dir(A), "ks_orbital_info.out")
        )
        weights = np.zeros(ks_info.shape[0])
        if restart_idx == 0: # Find HOMO states
            kso_idxs = aims_parser.find_homo_kso_idxs(ks_info)
        elif restart_idx == 1:
            kso_idxs = aims_parser.find_lumo_kso_idxs(ks_info)
        else:
            raise ValueError("...")

        # Add a weighting of 1.0
        for kso_idx in kso_idxs:  # these are 1-indexed in AIMS
            weights[kso_idx - 1] = 1.0

        # Save these to file so AIMS can read them in
        np.savetxt(os.path.join(scf_dir(A), "ks_orbital_weights.in"), weights)

    # And the general settings for all calcs
    aims_kwargs = base_aims_kwargs.copy()
    aims_kwargs.update(ri_kwargs)

    # Run the RI fitting procedure in AIMS
    aims_calc.run_aims_array(
        calcs=calcs,
        aims_path=aims_path,
        aims_kwargs=aims_kwargs,
        sbatch_kwargs=sbatch_kwargs,
        run_dir=scf_dir,
        restart_idx=restart_idx,   # Restart from a converged SCF
        copy_files=["ks_orbital_weights.in"],  # Copy the KS weights into the restart dir
    )

sbatch: [ESTIMATION] The estimated cost of this job is CHF 11.20
sbatch: [CAPPING]    All users of the account cosmo-erc have consumed 117.07 CHF


Submitted batch job 11396511
Submitted batch job 11396613


In [12]:
for restart_idx in [0, 1]:
    # Running this code will ensure that all calculations have finished
    # Should take < 30 seconds (1 node, 10 cores, on 'jed' HPC)
    all_finished = False
    while not all_finished:
        calcs_finished = []
        for A in idxs:
            aims_out_path = os.path.join(ri_dir(A, restart_idx), "aims.out")
            if os.path.exists(aims_out_path):
                with open(aims_out_path, "r") as f:
                    calcs_finished.append("Leaving FHI-aims." in f.read())  # AIMS finished
            else:
                calcs_finished.append(False)
        all_finished = np.all(calcs_finished)

    print("All RI calculations finished!")

All RI calculations finished!
All RI calculations finished!


In [18]:
for restart_idx in [0, 1]:
    # Process AIMS results and print density fitting error
    # Run in serial: takes approx 0.5 seconds per structure
    df_errors = []
    for A, frame in zip(idxs, frames):
        aims_parser.process_aims_ri_results(
            frame=frame,
            aims_output_dir=ri_dir(A, restart_idx),  # includes the restart idx
            process_what=["coeffs", "ovlp"],
            structure_idx=A,
        )
        calc_info = io.unpickle_dict(os.path.join(processed_dir(A, restart_idx), "calc_info.pickle"))
        df_errors.append(calc_info["df_error_percent"]["total"])
    print("Mean density fitting error across all structures (%):", np.mean(df_errors))

Mean density fitting error across all structures (%): 0.3762254669830918
Mean density fitting error across all structures (%): 2.6293376433963145


## Build Structural Descriptors 

* Here we construct $\lambda$-SOAP equivariant descriptors for each structure
* First, find the angular orders present in the decomposition of the target
  scalar field onto the RI basis

In [20]:
for restart_idx in [0, 1]:
    # The basis set definition should is consistent for all atoms in the dataset,
    # given consistent AIMS settings. Print this definiton from the parsed outputs
    # from one of the structures
    basis_set = io.unpickle_dict(
        os.path.join(processed_dir(A, restart_idx), "calc_info.pickle")
    )["basis_set"]

    print(basis_set["def"])

{'lmax': {'O': 8, 'H': 4}, 'nmax': {('O', 0): 9, ('O', 1): 10, ('O', 2): 9, ('O', 3): 8, ('O', 4): 6, ('O', 5): 4, ('O', 6): 4, ('O', 7): 2, ('O', 8): 1, ('H', 0): 9, ('H', 1): 7, ('H', 2): 6, ('H', 3): 3, ('H', 4): 1}}
{'lmax': {'O': 8, 'H': 4}, 'nmax': {('O', 0): 9, ('O', 1): 10, ('O', 2): 9, ('O', 3): 8, ('O', 4): 6, ('O', 5): 4, ('O', 6): 4, ('O', 7): 2, ('O', 8): 1, ('H', 0): 9, ('H', 1): 7, ('H', 2): 6, ('H', 3): 3, ('H', 4): 1}}


The maximum angular order here is $l = 8$. This should be reflected in the
settings used to generate the equivariant descriptor - specifically in
`cg_settings` in "settings.py".

In [ ]:
from settings import all_frames, rascal_settings, cg_settings

# Generate a rascaline SphericalExpansion (2 body) representation. As we want to
# retain the original structure indices, we are going to pass * all * of the
# 1000 frames to rascaline, but only compute for the subset of structures in
# `idxs`.
calculator = rascaline.SphericalExpansion(**rascal_settings["hypers"])
nu_1_tensor = calculator.compute(
    all_frames, 
    selected_samples=Labels(names=["structure"], values=idxs.reshape(-1, 1)),
    **rascal_settings["compute"],
)
nu_1_tensor = nu_1_tensor.keys_to_properties("species_neighbor")

# Build a lambda-SOAP descriptor by a CLebsch-Gordan combination
lsoap = clebsch_gordan.lambda_soap_vector(nu_1_tensor, **cg_settings)

# Check the resulting structure indices match those in `idxs`
assert np.all(
    np.sort(idxs)
    == metatensor.unique_metadata(lsoap, "samples", "structure").values.reshape(-1)
)

# Split into per-structure TensorMaps and save into separate directories.
# This is useful for batched training.
for A in idxs:
    lsoap_A = metatensor.slice(
        lsoap,
        "samples",
        labels=Labels(names="structure", values=np.array([A]).reshape(-1, 1)),
    )
    for restart_idx in [0, 1]:
        metatensor.save(os.path.join(processed_dir(A, restart_idx), "lsoap.npz"), lsoap_A)

The settings used to build the descriptor from an `.xyz` file, as well as build
the desired target property (the real-space scalar field) from the model
prediction need to be stored so that the perform can make a true end-to-end
prediction.

In the `rholearn` module "predictor.py", the functions `descriptor_builder` and
`target_builder` are implemented to perform these transformations on the input
and output side of the model, respectively. Both take a
variable input that depends on the structure being predicted on, and some
settings for performing the relevant transformations of the data. The key
physics-related settings used in predicting on an unseen structure shouldbe the
same as were used for generating the data the model was trained on.

Here we store the relevant settings in dictionaries, which will be used to
initialize the ML model later.

## Build `dataset`

* For cross-validation we create a train-test-val split of the data.
* Data is stored on the per-structure basis to help with mini-batching in training.

In [4]:
from settings import crossval_settings, ml_settings, run_dir, torch_settings

In [5]:
# Perform a train/test/val split of structure idxs
train_idxs, test_idxs, val_idxs = data.group_idxs(
    idxs=idxs,
    n_groups=crossval_settings["n_groups"],
    group_sizes=crossval_settings["group_sizes"],
    shuffle=crossval_settings["shuffle"],
    seed=crossval_settings["seed"],
)
print(
    "num train_idxs:",
    len(train_idxs),
    "   num test_idxs:",
    len(test_idxs),
    "   num val_idxs:",
    len(val_idxs),
)
np.savez(
    os.path.join(run_dir, "idxs.npz"),
    idxs=idxs,
    train_idxs=train_idxs,
    test_idxs=test_idxs,
    val_idxs=val_idxs,
)

num train_idxs: 120    num test_idxs: 40    num val_idxs: 40


* Visualise the dataset again, this time colored by its cross-validation category

In [6]:
crossval_category = lambda A: 0 if A in train_idxs else (1 if A in test_idxs else 2)

chemiscope.show(
    frames,
    properties={
        "Mean O-H bond length, Angstrom": [
            np.mean([frame.get_distance(0, 1), frame.get_distance(0, 2)])
            for A, frame in zip(idxs, frames)
        ],
        "H-O-H angle, degrees": [
            frame.get_angle(1, 0, 2) for A, frame in zip(idxs, frames)
        ],
        "0: train, 1: test, 2: val": [crossval_category(A) for A, frame in zip(idxs, frames)]
    },
)

ChemiscopeWidget(value='{"meta": {"name": " "}, "structures": [{"size": 3, "names": ["O", "H", "H"], "x": [0.0…

In [ ]:
# Construct dataset, defining callables to access the input, output, and overlap
# data from the structure indices
rho_data = data.RhoData(
    idxs=idxs,
    in_path=lambda A: os.path.join(processed_dir(A), "lsoap.npz"),
    out_path=lambda A: os.path.join(processed_dir(A), "ri_coeffs.npz"),
    aux_path=lambda A: os.path.join(processed_dir(A), "ri_ovlp.npz"),
    keep_in_mem=ml_settings["loading"]["train"]["keep_in_mem"],
    **torch_settings,
)

## Initialize model

* As typically the magnitude of the invariant RI-coefficients are many orders of
  magnitude larger than the covariant coefficients, it often helps model
  training if the invariant block models predict a baselined quantity. The
  baseline is then added back in on the TensorMap level before loss evaluation,
  acting as a kind of non-learnable bias.
* We can compute the mean features of the invariant blocks of the training data
  and initialize the model with this. Alternatively, one could use free-atom
  superpositions of the scalar field of interest as the baseline.

In [ ]:
if ml_settings["model"]["use_invariant_baseline"]:
    invariant_baseline = rho_data.get_invariant_means(
        idxs=train_idxs, which_data="output"
    )
else:
    invariant_baseline = None

for block in invariant_baseline:
    print(block)
    print(block.values)

* We can also calculate the standard deviation of the training data. This is
  defined relative to this invariant baseline.

In [ ]:
stddev = rho_data.get_standard_deviation(
    idxs=train_idxs,
    which_data="output",
    invariant_baseline=invariant_baseline,
    use_overlaps=True,
)
np.savez(os.path.join(run_dir, "stddev.npz"), stddev=stddev.detach().numpy())
stddev

* In order to perform end-to-end predictions, we also need to initialize the
  model with the settings used to build the equivariant descriptor and final
  target property. These will be the same as above, used to generate the
  training data.

In [ ]:
from settings import rascal_settings, cg_settings, base_aims_kwargs

# For descriptor building, we need to store the rascaline settings for
# generating a SphericalExpansion and performing Clebsch-Gordan combinations.
# The `descriptor_builder` function in "predictor.py" contains the 'recipe' for
# using these settings to transform an ASE Atoms object.
descriptor_kwargs = {
    "rascal_settings": rascal_settings,
    "cg_settings": cg_settings,
}

# For target building, the base AIMS settings need to be stored, along with the
# basis set definition.
basis_set = io.unpickle_dict(os.path.join(processed_dir(A), "calc_info.pickle"))["basis_set"]

target_kwargs = {
    "aims_kwargs": {**base_aims_kwargs},
    "basis_set": {**basis_set},
}

In [ ]:
# Initialize model
model = models.RhoModel(
    # Standard model architecture
    model_type=ml_settings["model"]["model_type"],  # "linear" or "nonlinear"
    input=rho_data[idxs[0]][1],   # example input data for init metadata
    output=rho_data[idxs[0]][2],  # example output data for init metadata
    bias_invariants=ml_settings["model"]["bias_invariants"],

    # Architecture settings if using a nonlinear base model
    hidden_layer_widths=ml_settings["model"].get("hidden_layer_widths"),
    activation_fn=ml_settings["model"].get("activation_fn"),
    bias_nn=ml_settings["model"].get("bias_nn"),

    # Settings for descriptor/target building
    descriptor_kwargs=descriptor_kwargs,
    target_kwargs=target_kwargs,

    # Torch tensor settings
    **torch_settings
)

## Initialize training objects: loaders, loss, optimizer

In [ ]:
# Construct dataloaders
train_loader = data.RhoLoader(
    rho_data,
    idxs=train_idxs,
    get_aux_data=True,  # load the overlap matrix
    batch_size=ml_settings["loading"]["train"]["batch_size"],
)
test_loader = data.RhoLoader(
    rho_data,
    idxs=test_idxs,
    get_aux_data=True,   # load the overlap matrix
    batch_size=ml_settings["loading"]["test"]["batch_size"],
)
# For the validation set, we want to evaluate the performance of the model
# against the real-space scalar field (requires calling AIMS), so the overlaps
# do not need to be loaded.
val_loader = data.RhoLoader(
    rho_data,
    idxs=val_idxs,
    get_aux_data=False,
    batch_size=None,
)

In [ ]:
# Initialize loss fxn and optimizer (don't use a scheduler for now)
loss_fn = loss.L2Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = None

* Finally, let's update the model with some settings needed to rebuild the
  real-space density from RI-coefficients in AIMS, needed for computing the L1 error.

In [ ]:
# Settings specific to RI rebuild procedure
ri_kwargs = {
    # Force no SCF
    "sc_iter_limit": 0,
    "postprocess_anyway": True,
    "ri_fit_assume_converged": True,
    # What we want to do
    "ri_fit_rebuild_from_coeffs": True,
    # What we want to output
    "ri_fit_write_rebuilt_field": True,
    "ri_fit_write_rebuilt_field_cube": True,
    "output": ["cube ri_fit"],  # needed for cube files
}

# Update the AIMS and SBATCH kwargs
tmp_aims_kwargs = {**model.target_kwargs["aims_kwargs"]}
tmp_aims_kwargs.update(ri_kwargs)

# Settings for slurm
sbatch_kwargs = {
    "job-name": "h2o-pred",
    "nodes": 1,
    "time": "01:00:00",
    "mem-per-cpu": 2000,
    "partition": "standard",
    "ntasks-per-node": 10,
}
model.update_target_kwargs(
    {
        "aims_path": aims_path,
        "aims_kwargs": tmp_aims_kwargs,
        "sbatch_kwargs": sbatch_kwargs,
    }
)

## Run model training

* We train the model by gradient descent, evaluating the loss of the prediction
  against the density-fitted quantity. As the learning target is the real-space
  density, which has been expanded on a non-orthogonal basis, and not just the
  RI coefficients, the overlap matrices must be used in loss evaluation:

* For a structure $A$, the L2 loss of the predicted scalar field relative to the **RI
  approximated scalar field** is defined as:

  * $ \mathcal{L}_A^{\text{L2}} = \left( \textbf{c}_A^{\text{ML}} - \textbf{c}_A^{\text{RI}} \right) \ . \ \hat{S}_A \ . \ \left( \textbf{c}_A^{\text{ML}} - \textbf{c}_A^{\text{RI}} \right)  $


* And L1 (MAE) error of the of the ML prediction relative to the **true QM
  scalar field** is defined as: 

  * $ \mathcal{L}_A^{\text{L1}} = \frac{1}{\int d\textbf{r} \rho_A^{\text{QM}} (\textbf{r})} \int d\textbf{r} \vert \rho_A^{\text{ML}} (\textbf{r}) - \rho_A^{\text{QM}} (\textbf{r}) \vert $
  
* While the L2 loss is used to train the model, we will also periodically
  calculate the L1 loss to assess the performance of the model relative to the
  actual true QM scalar field.

In [ ]:
from rholearn import train
from settings import run_dir, ml_settings

# Define a directory to save model checkpoints
chkpt_dir = os.path.join(run_dir, "checkpoints")
if not os.path.exists(chkpt_dir):
    os.makedirs(chkpt_dir)

# Define a dir to save predictions - needs to be a callable that takes tructure
# idx as an argument
if not os.path.exists(os.path.join(run_dir, "predictions")):
    os.makedirs(os.path.join(run_dir, "predictions"))

def pred_dir(A):
    return os.path.join(run_dir, "predictions", f"{A}")

# Define a log file for writing losses at each epoch
log_path = os.path.join(run_dir, "training.log")
io.log(log_path, "# epoch train_L2_loss test_L2_loss train_L1_error test_L1_error time")

# Run training loop
for epoch in range(1, ml_settings["training"]["n_epochs"] + 1):
    t0 = time.time()
    # Training step
    train_loss_epoch, test_loss_epoch = train.training_step(
        train_loader=train_loader,
        test_loader=test_loader,
        model=model,
        loss_fn=loss_fn,
        optimizer=optimizer,
        scheduler=scheduler,
        check_args=True
        if epoch == 1
        else False,  # Switch off metadata checks after 1st epoch
    )

    # Calculate also the L1 error of train/test predictions against the
    # real-space QM quantity. Store values of -1 if not evaluating the L1 error
    # this epoch
    mean_maes = {"train": -1, "test": -1}
    if (epoch - 1) % ml_settings["validation"]["interval"] == 0:

        mean_maes = {"train": [], "test": []}
        
        # Get frames and make prediction
        tmp_idxs = np.concatenate([train_idxs, test_idxs])
        tmp_frames = [all_frames[A] for A in tmp_idxs]
        pred_coeffs, pred_fields = model.predict(
            structure_idxs=tmp_idxs,
            frames=tmp_frames,
            save_dir=pred_dir,
        )

        # Evaluate mean L1 Error
        for A, pred_field in zip(tmp_idxs, pred_fields):
            # Get grids and check they're the same in the SCF and ML directories
            grid = np.loadtxt(os.path.join(ri_dir(A), "partition_tab.out"))
            assert np.allclose(
                grid,
                np.loadtxt(os.path.join(pred_dir(A), "partition_tab.out")),
            )

            # Get L1 error vs real-space QM scalar field
            target_field = np.loadtxt(os.path.join(ri_dir(A), "rho_ref.out"))
            mae = aims_parser.get_percent_mae_between_fields(
                input=pred_field,
                target=target_field,
                grid=grid,
            )
            if A in train_idxs:
                mean_maes["train"].append(mae)
            elif A in test_idxs:
                mean_maes["test"].append(mae)
        for category in ["train", "test"]:
            mean_maes[category] = np.mean(mean_maes[category])

    dt = time.time() - t0
    print(
        f"epoch {epoch}  ",
        f"train_L2_loss {np.round(train_loss_epoch.detach().numpy(), 5)}  "
        f"test_L2_loss {np.round(test_loss_epoch.detach().numpy(), 5)}  "
        f"time {dt}",
    )
    io.log(
        log_path,
        f"{epoch} {train_loss_epoch} {test_loss_epoch} {mean_maes['train']} {mean_maes['test']} {dt}",
    )
    if (epoch - 1) % ml_settings["training"]["save_interval"] == 0:
        torch.save(model, os.path.join(chkpt_dir, f"model_{epoch}.pt"))
        torch.save(optimizer.state_dict, os.path.join(chkpt_dir, f"opt_{epoch}.pt"))

In [ ]:
# Load training log
losses = np.loadtxt(os.path.join(run_dir, "training.log"))

# Unpack data from each row
epochs, train_loss, test_loss, train_mae, test_mae, times = losses.T

# Plot the various errors
fig, ax = plt.subplots()
ax.plot(epochs, train_loss, label="L2 Loss (ML/RI), train")
ax.plot(epochs, test_loss, label="L2 Loss (ML/RI), test")
ax.scatter(
    epochs[np.where(train_mae != -1)],
    train_mae[np.where(train_mae != -1)] / 100,
    label="L1 Error (ML/QM), train",
    marker=".",
)
ax.scatter(
    epochs[np.where(test_mae != -1)],
    test_mae[np.where(test_mae != -1)] / 100,
    label="L1 Error (ML/QM), test",
    marker=".",
)

ax.set_xlabel("Epochs")
ax.set_ylabel("Error per structure")
ax.set_yscale("log")
ax.set_title("HOMO-learning w/ NN | water monomer | AIMS cluster calculation")
ax.legend()

* This process is slow in a notebook - better to make use of HPC resources! The
  training procedure can essentially be copied to a seperate python script and
  run using a job scheduler - see "run_training.py".
* We can load a model that has been pre-trained and validate its performance
* Here we load a model trained to over 1500 epochs, only a dataset of only 10
  water molecules.

## Evaluate model performance on validation set



In [ ]:
# Load pre-trained model and the training log file
model = torch.load(os.path.join(run_dir, "trained_models", "model_1611.pt"))
log_file = np.loadtxt(os.path.join(run_dir, "trained_models", "training.log"))
log_file

In [ ]:
# Unpack data from each row
epochs, train_loss, test_loss, train_mae, test_mae, val_mae, times = log_file.T

# Plot the various errors
fig, ax = plt.subplots()
ax.plot(epochs, train_loss, label="L2 Loss (ML/RI), train")
ax.plot(epochs, test_loss, label="L2 Loss (ML/RI), test")
ax.scatter(
    epochs[np.where(train_mae != -1)],
    train_mae[np.where(train_mae != -1)] / 100,
    label="L1 Error (ML/QM), train",
    marker=".",
)
ax.scatter(
    epochs[np.where(test_mae != -1)],
    test_mae[np.where(test_mae != -1)] / 100,
    label="L1 Error (ML/QM), test",
    marker=".",
)

ax.set_xlabel("Epochs")
ax.set_ylabel("Error per structure")
ax.set_yscale("log")
ax.set_title("HOMO-learning w/ NN | water monomer | AIMS cluster calculation")
ax.legend()

## End-to-end prediction on the validation set

* Now let's make an end-to-end prediction (from xyz -> real-space scalar field)
  on the validation set

In [ ]:
# Load the validation frames as ASE Atoms objects
val_frames = [all_frames[A] for A in val_idxs]

def val_dir(A):
    return os.path.join(run_dir, "validation", f"{A}")

# Make predictions for the validation set. Note: we could predict from the
# descriptors that are already constructed here, but do so from ASE frames for
# demonstrative purposes
pred_coeffs, pred_fields = model.predict(structure_idxs=val_idxs, frames=val_frames, save_dir=val_dir)

In [ ]:
# Calculate density fitting error of predictions relative to SCF-converged
# real-space scalar field
val_df_errors = {}
for A, pred_field in zip(val_idxs, pred_fields):
    # Get grids and check they're the same in the SCF and ML directories
    grid = np.loadtxt(os.path.join(ri_dir(A), "partition_tab.out"))
    assert np.allclose(grid, np.loadtxt(os.path.join(val_dir(A), "partition_tab.out")))

    # Get DF error
    target_field = np.loadtxt(os.path.join(ri_dir(A), "rho_ref.out"))  # QM scalar field
    df_error = aims_parser.get_percent_mae_between_fields(
        input=pred_field,
        target=target_field,
        grid=grid,
    )
    print(f"Val structure {A}, DF error (%): {df_error}")
    val_df_errors[A] = df_error

* Let's visualize these predictions using the cube file outputs
* Pick one validation structure and vizualize the target QM and predicted ML
  HOMO scalar fields

In [ ]:
# Visualize the predicted density
A = val_idxs[0]

qm_cube = os.path.join(ri_dir(A), "rho_ref.cube")
ml_cube = os.path.join(val_dir(A), "rho_rebuilt.cube")

for cube_file in [qm_cube, ml_cube]:
    v = py3Dmol.view()
    v.addModelsAsFrames(open(cube_file, "r").read(), "cube")
    v.setStyle({"stick": {}})
    v.addVolumetricData(
        open(cube_file, "r").read(),
        "cube",
        {"isoval": 0.001, "color": "blue", "opacity": 0.8},
    )
    v.show()

* Let's calculate the difference between ML and QM RI-coefficients, and use
  these 'delta-coeffs' to reconstruct a delta-scalar field

In [ ]:
# First define a new directory to store the delta HOMOs
if not os.path.exists(os.path.join(run_dir, "delta")):
    os.mkdir(os.path.join(run_dir, "delta"))

def delta_dir(A):
    return os.path.join(run_dir, "delta", f"{A}")

In [ ]:
# Retrieve the target coeffs
target_coeffs = [
    metatensor.load(os.path.join(processed_dir(A), "ri_coeffs.npz"))
    for A in val_idxs
]

# Calculate the delta coeffs
delta_coeffs = [
    metatensor.subtract(pred_coeff, target_coeff) for pred_coeff, target_coeff in zip(pred_coeffs, target_coeffs)
]

# Build the delta-HOMO from RI coefficients using AIMS
delta_fields = predictor.target_builder(
    structure_idxs=val_idxs, 
    frames=val_frames, 
    predictions=delta_coeffs, 
    save_dir=delta_dir, 
    **model.target_kwargs
)

In [ ]:
# Visualize the delta-HOMO of one of the validation structures
A = val_idxs[0]
delta_cube = os.path.join(delta_dir(A), "rho_rebuilt.cube")

for cube_file in [delta_cube]:
    v = py3Dmol.view()
    v.addModelsAsFrames(open(cube_file, "r").read(), "cube")
    v.setStyle({"stick": {}})
    v.addVolumetricData(
        open(cube_file, "r").read(),
        "cube",
        {"isoval": 0.001, "color": "blue", "opacity": 0.8},
    )
    v.show()